In [11]:
#!pip install opencv-python
!pip install --upgrade --force-reinstall albumentations
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
import albumentations as A

Defaulting to user installation because normal site-packages is not writeable
  Using cached albumentations-2.0.5-py3-none-any.whl.metadata (41 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 330.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 330.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 330.3 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 231.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Akoba\\AppData\\Roaming\\Python\\Python311\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



ModuleNotFoundError: No module named 'albumentations'

In [ ]:

# Define paths
base_dir = "C:/Users/Akoba/Desktop/START up/COMPUTER-AIDED-VISION-FOR-PLANT-DISEASE-DETECTION"
data_dir = os.path.join(base_dir, "Data/PlantVillage")
metadata_csv = os.path.join(base_dir, "Data/PlantVillage/metadata.csv")

In [ ]:

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:

# Create directories for saving cleaned data
cleaned_dir = os.path.join(base_dir, "Data/PlantVillage/cleaned")
os.makedirs(cleaned_dir, exist_ok=True)

In [ ]:
df = pd.read_csv(metadata_csv)
print("Dataset Overview:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print(f"\nTotal Images: {len(df)}")
print(f"Total Classes: {df['label'].nunique()}")
print("\nClasses:")
print(df['label'].unique())

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x="label", data=df)
plt.xticks(rotation=90)
plt.title("Class Distribution")
plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.show()

In [ ]:
# Print class counts
print("\nClass Counts:")
print(df["label"].value_counts())

In [ ]:
def check_image_quality(img_path):
    try:
        # Load image with OpenCV
        img = cv2.imread(img_path)
        if img is None:
            return False, "Corrupted"
        
        # Convert to grayscale for blur detection
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect blur using Laplacian variance (low variance = blurry)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        if laplacian_var < 100:  # Threshold for blur
            return False, "Blurry"
        
        # Check brightness (too dark or too bright)
        brightness = np.mean(img)
        if brightness < 30 or brightness > 225:  # Thresholds for brightness
            return False, "Too dark/bright"
        
        return True, "Good"
    except Exception as e:
        return False, f"Error: {str(e)}"

# Sample 100 images for quality check
sample_df = df.sample(n=min(100, len(df)), random_state=42)
quality_results = []
for img_path in sample_df["image_path"]:
    is_good, reason = check_image_quality(img_path)
    quality_results.append({"image_path": img_path, "is_good": is_good, "reason": reason})

quality_df = pd.DataFrame(quality_results)
print("\nImage Quality Check (Sample):")
print(quality_df["reason"].value_counts())

# Filter out bad images
bad_images = quality_df[quality_df["is_good"] == False]["image_path"]
print(f"\nNumber of bad images in sample: {len(bad_images)}")